# Gamma MAP Filter

The objective is to filter input images with Gamma MAP.

In [19]:
import os
import cv2
from scipy.ndimage import zoom
import numpy as np

The Gamma MAP Filter code was downloaded from https://github.com/mortcanty/SARDocker/blob/master/tutorialsar.ipynb. It was modified to run with Python 3. The original version was written for Python 2.0.

In [21]:
def gamma_filter(k, inimage, rows, cols, m):    
    def get_windex(j, cols):     
        windex = np.zeros(49, dtype=int)
        six = np.array([0, 1, 2, 3, 4, 5, 6])      
        windex[0:7] = (j-3)*cols + six
        windex[7:14] = (j-2)*cols + six
        windex[14:21] = (j-1)*cols + six
        windex[21:28] = (j)*cols + six 
        windex[28:35] = (j+1)*cols + six 
        windex[35:42] = (j+2)*cols + six
        windex[42:49] = (j+3)*cols + six
        return windex  

    templates = np.zeros((8, 7, 7), dtype=int)
    for j in range(7):
        templates[0, j, 0:3] = 1
    templates[1, 1, 0] = 1
    templates[1, 2, :2] = 1
    templates[1, 3, :3] = 1
    templates[1, 4, :4] = 1
    templates[1, 5, :5] = 1
    templates[1, 6, :6] = 1
    templates[2] = np.rot90(templates[0])
    templates[3] = np.rot90(templates[1])
    templates[4] = np.rot90(templates[2])
    templates[5] = np.rot90(templates[3])
    templates[6] = np.rot90(templates[4])
    templates[7] = np.rot90(templates[5])
    
    tmp = np.zeros((8, 21), dtype=int)
    for i in range(8):
        tmp[i, :] = np.where(templates[i].ravel())[0] 
    templates = tmp
    
    edges = np.zeros((4, 3, 3), dtype=int)
    edges[0] = [[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]
    edges[1] = [[0, 1, 1], [-1, 0, 1], [-1, -1, 0]]
    edges[2] = [[1, 1, 1], [0, 0, 0], [-1, -1, -1]]
    edges[3] = [[1, 1, 0], [1, 0, -1], [0, -1, -1]]        
    result = np.copy(inimage[k])
    arr = inimage[k].ravel()
    zf = 3. / 7
    for j in range(3, rows-3):
        if j % 50 == 0:
            print('band %i  row %i' % ((k+1), j))  # Updated print statement
        windex = get_windex(j, cols)
        for i in range(3, cols-3):
            g = inimage[k, j, i]            
            wind = np.reshape(arr[windex], (7, 7))
            # 3x3 compression             
            w = zoom(wind, zf, order=1, prefilter=False)              
            # get appropriate edge mask
            es = [np.sum(edges[p] * w) for p in range(4)]
            idx = np.argmax(es)  
            if idx == 0:
                if np.abs(w[1, 1] - w[1, 0]) < np.abs(w[1, 1] - w[1, 2]):
                    edge = templates[0]
                else:
                    edge = templates[4]
            elif idx == 1:
                if np.abs(w[1, 1] - w[2, 0]) < np.abs(w[1, 1] - w[0, 2]):
                    edge = templates[1]
                else:
                    edge = templates[5]                
            elif idx == 2:
                if np.abs(w[1, 1] - w[0, 1]) < np.abs(w[1, 1] - w[2, 1]):
                    edge = templates[6]
                else:
                    edge = templates[2]  
            elif idx == 3:
                if np.abs(w[1, 1] - w[0, 0]) < np.abs(w[1, 1] - w[2, 2]):
                    edge = templates[7]
                else:
                    edge = templates[3] 
            wind = wind.ravel()[edge] 
            var = np.var(wind)
            if var > 0: 
                mu = np.mean(wind)  
                alpha = (1 + 1.0 / m) / (var / mu**2 - 1 / m)
                if alpha < 0:
                    alpha = np.abs(alpha)
                a = mu * (alpha - m - 1)
                x = (a + np.sqrt(4 * g * m * alpha * mu + a**2)) / (2 * alpha)        
                result[j, i] = x
            windex += 1  
                   
    return result

In [20]:
input_dir = '../results/selected_images'
input_files = []
for file in os.listdir(input_dir):
    name, ext = os.path.splitext(file)
    if ext == '.png':
        input_files.append(os.path.join(input_dir, file))
input_files

['../results/selected_images/sentinel-1_image_VH_2018-08-12.png',
 '../results/selected_images/sentinel-1_image_VH_2018-12-10.png',
 '../results/selected_images/sentinel-1_image_VH_2018-10-11.png',
 '../results/selected_images/sentinel-1_image_VH_2019-04-09.png',
 '../results/selected_images/sentinel-1_image_VH_2019-03-04.png',
 '../results/selected_images/sentinel-1_image_VH_2018-09-05.png',
 '../results/selected_images/sentinel-1_image_VH_2019-01-03.png',
 '../results/selected_images/sentinel-1_image_VH_2019-02-08.png',
 '../results/selected_images/sentinel-1_image_VH_2018-11-04.png']

In [25]:
output_dir = '../results/gamma_map'
os.makedirs(output_dir, exist_ok=True)

output_files = [] 
for file in input_files:
    file = os.path.relpath(file, input_dir)
    name, ext = os.path.splitext(file)
    output_files.append(os.path.join(output_dir, file))

kernel_size = 5
input_output_mapping = list(zip(input_files, output_files))
for input_file, output_file in input_output_mapping:
    print(f'Gamma_MAP(input_file={input_file}, output_file={output_file})')
    if os.path.exists(output_file):
        print(f'>>> Skipping step... This file exists.')
    else:
        image = cv2.imread(input_file, cv2.IMREAD_GRAYSCALE) 
        rows, cols = image.shape
        image = np.array([image])
        k = 0 # only one channel
        m = 1 
        filtered_image = gamma_filter(0, image, rows, cols, m)
        cv2.imwrite(output_file, filtered_image)        

Gamma_MAP(input_file=../results/selected_images/sentinel-1_image_VH_2018-08-12.png, output_file=../results/gamma_map/sentinel-1_image_VH_2018-08-12.png)


/Users/jimuelcelestejr/opt/anaconda3/envs/sar-speckle-noise-filtering/lib/python3.6/site-packages/ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in ubyte_scalars
/Users/jimuelcelestejr/opt/anaconda3/envs/sar-speckle-noise-filtering/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: overflow encountered in ubyte_scalars
/Users/jimuelcelestejr/opt/anaconda3/envs/sar-speckle-noise-filtering/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: overflow encountered in ubyte_scalars
/Users/jimuelcelestejr/opt/anaconda3/envs/sar-speckle-noise-filtering/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: overflow encountered in ubyte_scalars


band 1  row 50
band 1  row 100
band 1  row 150
band 1  row 200
band 1  row 250
band 1  row 300
band 1  row 350
band 1  row 400
band 1  row 450
band 1  row 500
band 1  row 550
band 1  row 600
band 1  row 650
band 1  row 700
band 1  row 750
band 1  row 800
band 1  row 850
band 1  row 900
band 1  row 950
band 1  row 1000
band 1  row 1050
band 1  row 1100
band 1  row 1150
band 1  row 1200
band 1  row 1250
band 1  row 1300
band 1  row 1350
band 1  row 1400
band 1  row 1450
band 1  row 1500
band 1  row 1550
band 1  row 1600
band 1  row 1650
band 1  row 1700
band 1  row 1750
band 1  row 1800
Gamma_MAP(input_file=../results/selected_images/sentinel-1_image_VH_2018-12-10.png, output_file=../results/gamma_map/sentinel-1_image_VH_2018-12-10.png)
band 1  row 50
band 1  row 100
band 1  row 150
band 1  row 200
band 1  row 250
band 1  row 300
band 1  row 350
band 1  row 400
band 1  row 450
band 1  row 500
band 1  row 550
band 1  row 600
band 1  row 650
band 1  row 700
band 1  row 750
band 1  row 800
